# Final Project
#### CSCI 608
#### Jacob Dodson

In [37]:
import pandas as pd
import altair as alt

### **<u>Problem Formulation:</u>**

#### ***1. Descriptive Question:*** What are the overall patterns of academic performance (G3) in Math and Portuguese, and how many students fall into “at-risk” performance categories (G3 < 10)?

#### ***2. Exploratory Question:*** Which external student factors, such as study time, family support, or alcohol use, show the strongest relationships with being academically at risk (G3 < 10)?

#### ***3. Predictive Question:*** Can we build a regression model that predicts whether a student is at risk of failing (G3 < 10) without using G1 or G2, based only on demographic, behavioral, and/or family features?

#### ***4. Inferential Question:*** Do students with strong family support, measured through high family relationship quality (famrel), parental cohabitation (Pstatus), and family educational support (famsup), have significantly higher final grades (G3) compared to students with weaker support, based on bootstrap confidence intervals?

### **<u>Data Collection:</u>**

#### Data was collected from this link: [Student Performance](https://archive.ics.uci.edu/dataset/320/student+performance) which is a Protuguese student performance dataset from students in secondary education (high school).

#### The datasets combines objective school records such as grades (G1, G2, G3) and absences, with a range of self-reported questionnaire data, including demographic, behavioral, family, educational support, and social/emotional variables.

### **<u>Data Preparation:</u>**

In [2]:
mat = pd.read_csv("student-mat.csv", sep=";")
por = pd.read_csv("student-por.csv", sep=";")

In [3]:
mat.head()

,school,sex,age,address,famsize,Pstatus,Medu,Fedu,Mjob,Fjob,...,famrel,freetime,goout,Dalc,Walc,health,absences,G1,G2,G3
0,GP,F,18,U,GT3,A,4,4,at_home,teacher,...,4,3,4,1,1,3,6,5,6,6
1,GP,F,17,U,GT3,T,1,1,at_home,other,...,5,3,3,1,1,3,4,5,5,6
2,GP,F,15,U,LE3,T,1,1,at_home,other,...,4,3,2,2,3,3,10,7,8,10
3,GP,F,15,U,GT3,T,4,2,health,services,...,3,2,2,1,1,5,2,15,14,15
4,GP,F,16,U,GT3,T,3,3,other,other,...,4,3,2,1,2,5,4,6,10,10


#### Math dataset has 395 student performance observations

In [5]:
por.head()

,school,sex,age,address,famsize,Pstatus,Medu,Fedu,Mjob,Fjob,...,famrel,freetime,goout,Dalc,Walc,health,absences,G1,G2,G3
0,GP,F,18,U,GT3,A,4,4,at_home,teacher,...,4,3,4,1,1,3,4,0,11,11
1,GP,F,17,U,GT3,T,1,1,at_home,other,...,5,3,3,1,1,3,2,9,11,11
2,GP,F,15,U,LE3,T,1,1,at_home,other,...,4,3,2,2,3,3,6,12,13,12
3,GP,F,15,U,GT3,T,4,2,health,services,...,3,2,2,1,1,5,0,14,14,14
4,GP,F,16,U,GT3,T,3,3,other,other,...,4,3,2,1,2,5,0,11,13,13


#### Portuguese dataset has 649 student performance observations

#### Both datasets are tidy. Each row corresponds to one student in a specific course and each column represents a single variable showing we have exactly one observation per row.

### **<u>Analysis:</u>**

#### ***1. Descriptive and Exploratory (data visualizations):***

In [6]:
mat_g3_mean = mat["G3"].mean()
por_g3_mean = por["G3"].mean()

In [7]:
mat_g3_median = mat["G3"].median()
por_g3_median = por["G3"].median()

In [8]:
mat_g3_counts = mat["G3"].value_counts().sort_index()
por_g3_counts = por["G3"].value_counts().sort_index()

In [22]:
mat_at_risk_count = (mat["G3"] < 10).sum()
por_at_risk_count = (por["G3"] < 10).sum()

In [23]:
mat_total = len(mat)
por_total = len(por)

In [24]:
mat_at_risk_pct = mat_at_risk_count / mat_total * 100
por_at_risk_pct = por_at_risk_count / por_total * 100

**Math (G3) Performance Stats**

In [25]:
print(f"Mean G3:, {mat_g3_mean:.2f}")

Mean G3:, 10.42


In [26]:
print(f"Median G3:, {mat_g3_median:.2f}")

Median G3:, 11.00


In [27]:
print("Distribution of G3")
mat_g3_counts

Distribution of G3


G3
0     38
4      1
5      7
6     15
7      9
8     32
9     28
10    56
11    47
12    31
13    31
14    27
15    33
16    16
17     6
18    12
19     5
20     1
Name: count, dtype: int64

In [30]:
print(f"At-risk math students (G3 < 10): {mat_at_risk_count}/{mat_total} " f"({mat_at_risk_pct:.1f}%)")

At-risk math students (G3 < 10): 130/395 (32.9%)


**Portuguese (G3) Performance Stats**

In [31]:
print(f"Mean G3:, {por_g3_mean:.2f}")

Mean G3:, 11.91


In [32]:
print(f"Median G3:, {por_g3_median:.2f}")

Median G3:, 12.00


In [33]:
print("Distribution of G3")
por_g3_counts

Distribution of G3


G3
0      15
1       1
5       1
6       3
7      10
8      35
9      35
10     97
11    104
12     72
13     82
14     63
15     49
16     36
17     29
18     15
19      2
Name: count, dtype: int64

In [36]:
print(f"At-risk Portuguese students (G3 < 10): {por_at_risk_count}/{por_total} " f"({por_at_risk_pct:.1f}%)")

At-risk Portuguese students (G3 < 10): 100/649 (15.4%)


In [38]:
mat['subject'] = 'Math'
por['subject'] = 'Portuguese'
df = pd.concat([mat, por])

In [57]:
df_plot = pd.concat([mat, por], ignore_index=True)

# Overlapped Histogram
overlap_hist = alt.Chart(df_plot).mark_bar(opacity=0.5).encode(
    x=alt.X("G3:Q", bin=alt.Bin(maxbins=20), title="Final Grade (G3)"),
    y=alt.Y("count()", title="Count"),
    color=alt.Color("subject:N", title="Subject"),
    tooltip=["subject", "G3"]
).properties(
    width=800,
    height=600,
    title="Overlapped Distribution of Final Grades (G3): Math vs Portuguese"
)
overlap_hist

alt.Chart(...)

In [58]:
df_plot["at_risk"] = df_plot["G3"] < 10

# Overlapped histogram of G3 by at-risk status
risk_hist = alt.Chart(df_plot).mark_bar(opacity=0.5).encode(
    x=alt.X("G3:Q", bin=alt.Bin(maxbins=20), title="Final Grade (G3)"),
    y=alt.Y("count()", title="Count"),
    color=alt.Color("at_risk:N",
                    title="At-Risk Status",
                    legend=alt.Legend(labelExpr="datum.label == 'True' ? 'At-Risk (G3 < 10)' : 'Not At-Risk'")),
    tooltip=["G3", "at_risk", "subject"]
).properties(
    width=800,
    height=600,
    title="Distribution of Final Grades (G3) by At-Risk Status"
)
risk_hist

alt.Chart(...)